# TP2
1. Um  programa imperativo pode ser descrito por um modelo do tipo Control Flow Automaton (CFA) como ilustrado no exemplo seguinte

<img src="automato.png" alt="autómato" style="width: 600px;"/>

Este programa implementa a multiplicação de dois inteiros a,b fornecidos como “input”, e com precisão limitada a $\,n\,$ bits (fornecido como parâmetro do programa). 

Note-se que:

    - Existe a possibilidade de alguma das operações do programa produzir um erro de “overflow”. 
    - Os nós do grafo representam ações  que actuam sobre os “inputs” do nó e produzem um output com as operações indicadas
    - Os ramos do grafo representam ligações que transferem o “output” de um nodo para o “input” do nodo seguinte. Esta transferência é condicionada pela satisfação da condição associada ao ramo


a. Construa um FOTS usando BitVector de tamanho $n$ que descreva o comportamento deste autómato. Para isso identifique as variáveis do modelo, o estado inicial e a relação de transição.

    
b. Verifique se $$\;\;\mathsf{P}\,\equiv\,(x*y + z = a*b)\;\;$$é um invariante deste comportamento.

In [10]:
from pysmt.shortcuts import And
from z3 import *

In [11]:
# FOTS = (SMT, vetor_vars, init, trans)
# SMT - LIA
# vetor_vars = ["a", "b", z]

def declare(i, n):
    state = {}
    state['pc'] = Int('pc'+str(i))
    state['x'] = BitVec('x'+str(i), n)
    state['y'] = BitVec('y'+str(i), n)
    state['z'] = BitVec('z'+str(i), n)
    
    return state


In [12]:
def init(state, a, b):
    return And(Equals(state['pc'], Int(0)), Equals(state['x'], a), Equals(state['y'], b), Equals(['z'], Int(0)))

In [13]:
def trans(curr, prox):
    t0 = And(
            Equals(curr['pc'], Int(0)),
            Equals(prox['pc'], Int(1)),
            Equals(curr['X'], prox['x']),
            Equals(curr['y'], prox['y']),
            Equals(curr['z'], prox['z']),
    )
    
    # y = 0
    t1 = And(
        Equals(sum(y), Int(0)),
        Equals(curr['pc'], Int(1)),
        Equals(prox['pc'], Int(5)),
        Equals(curr['x'], prox['x']),
        Equals(curr['y'], prox['y']),
        Equals(curr['z'], prox['z'])
    )
    
    # y != 0 ^ odd(y)
    t2 = And(
        Equals(sum(y), Int(0)),
        Equals(curr['y'][-1], Int(1)),
        Equals(curr['pc'], Int(1)),
        Equals(prox['pc'], Int(2)),
        Equals(curr['x'], prox['x']),
        Equals(prox['y'], curr['y'] -1),
        Equals(prox['z'], curr['z']+curr['x'])
    )
    
    # y != 0 ^ even(y)
    t3 = And(
        Equals(sum(y), Int(0)),
        Equals(curr['y'][-1], Int(0)),
        Equals(curr['pc'], Int(1)),
        Equals(prox['pc'], Int(3)),
        Equals(prox['x'], curr['x']*2),
        Equals(prox['y'], curr['y']/2),
        Equals(prox['z'], curr['z'])
    )
    
    # Q3 -> Q1
    t4 = And(
        Equals(curr['pc'], Int(3)),
        Equals(prox['pc'], Int(1)),
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z'])
    )
        
    #Q2 -> Q1
    t5 = And(
        Equals(curr['pc'], Int(2)),
        Equals(prox['pc'], Int(1)),
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['z'], curr['z'])
    )
     
    # tbd
    #t6 = 

In [16]:
def gera_traco(declare,init,trans,k):

    s = Solver()
    
    trace = [declare(i, 8) for i in range(k)]

    # adicionar o estado inicial
    s.add_assertion(init(trace[0]))
        
    for i in range(k - 1):
        s.add_assertion(trans(trace[i], trace[i+1]))
            
    if s.solve():
            
        for i in range(k):
            print("Passo ", i)
            for v in trace[i]:
                print(v, "=", s.get_value(trace[i][v]))
            print("----------------")

                
gera_traco(declare,init,trans,20)

AttributeError: 'Solver' object has no attribute 'add_assertion'